In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
from GAN_utils import *
from torchvision import models
from torchsummary import summary

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cpu")

In [3]:
labels = [4]
runs = 1
epochs = 3
batch_size = 1
noise_dim = 5
image_size = 8
loss = nn.BCELoss()
lr_gen = 0.3#0.03# 0.001
lr_disc = 0.01#0.001
n_qubits = 5 
ancillary_qubits = 1  
gen_n_layers = 6  
n_generators = 4 
model_type = ['Classical_linear', 'Quantum_linear']


digits, targets = datasets.load_digits(return_X_y=True)
rd, _ = resize_data(digits, targets, label = labels, image_size = image_size)
dataloader = torch.utils.data.DataLoader(rd, batch_size=batch_size, shuffle=True, drop_last=True)


discriminator = Discriminator(image_size).to(device)
qgenerator = QuantumGenerator(n_qubits, ancillary_qubits, gen_n_layers, n_generators, device).to(device)
generator = Generator(noise_dim).to(device)

results = run_model(dataloader, generator, discriminator, noise_dim, image_size, batch_size, loss, lr_gen, lr_disc, 
                    device, epochs, model_type[0])
results

2023-02-02 17:05:08.401124: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-02 17:05:08.401177: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 17:05:08.401195: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vt-bozzololu): /proc/driver/nvidia/version does not exist
2023-02-02 17:05:08.401465: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 3/3 [00:00<00:00,  7.42it/s, Generator loss=0.875, Discriminator loss=1.09]


In [ ]:
def generated_images(results):

    fig = plt.figure(figsize=(20, 10))
    outer = gridspec.GridSpec(len(results)//2, 2, wspace=0.1)

    for i, images in enumerate(results):
        inner = gridspec.GridSpecFromSubplotSpec(1, images.size(0), subplot_spec=outer[i])
        
        images = torch.squeeze(images, dim=1)
        for j, im in enumerate(images):

            ax = plt.Subplot(fig, inner[j])
            ax.imshow(im.numpy(), cmap="gray")
            ax.set_xticks([])
            ax.set_yticks([])
            if j==0:
                ax.set_title(f'Run {i+1}', loc='left', color = 'White')
            fig.add_subplot(ax)

    plt.show()

In [ ]:
generated_images(results[2])